In [ ]:
pip install pandas                            # Installing The library

In [ ]:
import os
os.getcwd()                                   # get current working directory

'/content'

In [ ]:
from bs4 import BeautifulSoup                 # for web scraping
import numpy as np                            # for linear algebra
import pandas as pd                           # for data processing
import requests as rq                         # for making HTTP requests

In [ ]:
# setting up global variables
global rating, Aircraft, Class, Travel_type, Route, Date, comfort, staff, food, entertainment, wifi, ground_service, vfm, recommend

verified=[]
rating=[]
feedback=[]
Aircraft=[]
Class=[]
Travel_type=[]
Route=[]
Date=[]
comfort=[]
staff=[]
food=[]
entertainment=[]
wifi=[]
ground_service=[]
vfm=[]
recommend=[]

In [ ]:
def value_fetcher(soup):                                                        # defining a function to process travel review HTML content
    rating.append(np.nan)                                                       # the lists of global variables are intialised by np.nan
    Aircraft.append(np.nan)
    Class.append(np.nan)
    Travel_type.append(np.nan)
    Route.append(np.nan)
    Date.append(np.nan)
    comfort.append(np.nan)
    staff.append(np.nan)
    food.append(np.nan)
    entertainment.append(np.nan)
    wifi.append(np.nan)
    ground_service.append(np.nan)
    vfm.append(np.nan)
    recommend.append(np.nan)

# structure of data
    for row in soup.find_all('tr'):                                             # iterates through all the table rows (tr) elements
        header=row.find('td',class_='review-rating-header').text                # title of the review
        value=row.find('td',class_='review-value')                              # values / elements of the review
        if header=='Aircraft':
            Aircraft[-1]=value.text
        elif header=='Type Of Traveller':
            Travel_type[-1]=value.text
        elif header=='Seat Type':
            Class[-1]=value.text
        elif header=='Route':
            Route[-1]=value.text
        elif header=='Date Flown':
            Date[-1]=value.text
        elif header=='Seat Comfort':
            comfort[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Cabin Staff Service':
            staff[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Food & Beverages':
            food[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Inflight Entertainment':
            entertainment[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Ground Service':
            ground_service[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Value For Money':
            vfm[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Recommended':
            recommend[-1]=value.text
        elif header=='Wifi & Connectivity':
            wifi[-1]=len(row.find_all('span',class_='star fill'))

In [ ]:
def fetcher(soup,tag,key,value):                                                # defining a function to extract data based on the predefined variables
    if soup.find(tag,{key:value}):
        return soup.find(tag,{key:value}).text
    return np.nan

In [ ]:
# Web Scraping
for i in range(1,340):                                                          # For loop for iteration through the website pages 1 to 340
    url=f'https://www.airlinequality.com/airline-reviews/vistara/page/{i}/'     # Webpage of Airline reviews
    page=rq.get(url).text                                                       # HTTP request to the URL
    soup=BeautifulSoup(page,'lxml')
    articles= soup.find_all('article',{'itemprop':'review'})                    # Finding review articles
    for article in articles:                                                    # Iterating through each review article
        rating.append(fetcher(article,'span','itemprop','ratingValue'))
        div=article.find('div',class_='tc_mobile')
        content=(div.find('div',class_='text_content').text.split('|'))
        if len(content)==2:
            verification,review=content
        else:
            verification =np.nan
            review=content[0]
        verified.append(verification)
        feedback.append(review)
        table=div.find('table',class_='review-ratings')
        value_fetcher(table)

In [ ]:
# constructing a Pandas DataFrame to organize the collected travel review data. Each aspect of the reviews is represented as a column

data=pd.DataFrame({'status':verified,'aircraft':Aircraft,'travel_type':Travel_type,'travel_class':Class,'route':Route,'date':Date,'seating_comfort':comfort,'staff_service':staff,'food_quality':food,'entertainment':entertainment,'wifi':wifi,'ground_service':ground_service,'value_for_money':vfm,'recommended':recommend,'overall_rating':rating[::2],'review':feedback})

In [ ]:
data.head(5)                                                                    # preview of the first 5 data records

,status,aircraft,travel_type,travel_class,route,date,seating_comfort,staff_service,food_quality,entertainment,wifi,ground_service,value_for_money,recommended,overall_rating,review
0,✅ Trip Verified,A321,Solo Leisure,Economy Class,Mumbai to Kolkata,September 2024,5.0,5.0,4.0,NaN,NaN,5.0,5.0,yes,8,It seems that on the early morning departure ...
1,✅ Trip Verified,NaN,Business,Business Class,Sydney to Delhi,May 2024,1.0,1.0,1.0,NaN,NaN,2.0,1.0,no,1,I flew from Sydney to Delhi on flight AI301...
2,✅ Trip Verified,NaN,Solo Leisure,Economy Class,Bangalore to Paris via Delhi,August 2024,2.0,1.0,2.0,1.0,NaN,1.0,1.0,no,1,Had a flight on 31 August from Bangalore to...
3,Not Verified,NaN,Solo Leisure,Economy Class,Sydney to Amritsar via Delhi,September 2024,NaN,NaN,NaN,NaN,NaN,1.0,1.0,no,1,The worst experience when I decided to book ...
4,✅ Trip Verified,NaN,Family Leisure,Economy Class,Delhi to Washington,September 2024,1.0,1.0,1.0,NaN,NaN,1.0,1.0,no,1,I was traveling from Delhi to Washington on...


In [ ]:
data['Airlines'] = 'Vistara'                                                    # Adding a column for the Airline name

In [ ]:
data.to_csv('Vistara.csv',index=False)                                          # Exporting the data

In [ ]:
data.head(5)

,status,aircraft,travel_type,travel_class,route,date,seating_comfort,staff_service,food_quality,entertainment,wifi,ground_service,value_for_money,recommended,overall_rating,review,Airlines
0,✅ Trip Verified,NaN,Solo Leisure,Economy Class,Italy to South Africa,July 2024,2,3,NaN,NaN,1,1,1,no,1,Let’s start off by saying I paid 2000€ for a ...,Vistara
1,✅ Trip Verified,NaN,Solo Leisure,Economy Class,Italy to South Africa,July 2024,2,3,NaN,NaN,1,1,1,no,1,Let’s start off by saying I paid 2000€ for a ...,Vistara
2,✅ Trip Verified,NaN,Solo Leisure,Economy Class,Italy to South Africa,July 2024,2,3,NaN,NaN,1,1,1,no,1,Let’s start off by saying I paid 2000€ for a ...,Vistara
3,✅ Trip Verified,NaN,Solo Leisure,Economy Class,Italy to South Africa,July 2024,2,3,NaN,NaN,1,1,1,no,1,Let’s start off by saying I paid 2000€ for a ...,Vistara
